# Pre Lab 09 : Eigenvalue Problems

## Objectives

This prelab is split into two parts.  In the first part we will explore calculating eigenvalues and eigenvectors.  In the second part we will prepare for the lab.  The **second part of the prelab is essential for completing the lab**.  In the lab this week we will solve the Schrödinger equation.  This can be confusing if we are unfamiliar with the basics of quantum mechanics. The prelab will lead us through the set up making the lab much easier (and faster).  **It is in your best interest to understand this setup prior to the lab.**

## Initialization

As always you should add initialization to the top of your notebook.

In [1]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

import scipy.interpolate as interp


## Eigenvalues and Eigenvectors


In homework 8, problem 2 we encountered a system of linear equations represented by the matrix
$$
\mathsf{A} = \begin{pmatrix}
4 & -7 & 3 \\
1 & 3 & -3 \\
3 & -29 & 21
\end{pmatrix}.$$

We should have seen that `scipy.linalg.solve` did provide a solution to the system of equations despite this system being linearly dependent.  Here we will study this matrix in more detail.

If a system of linear equations is not linearly independent, then the matrix representing this system should be singular.  Mathematically, what does this mean for the determinant of the matrix? 

If a matrix is singular, the determinate will be equal to 0

Construct the matrix as a NumPy array and numerically calculate and print its determinant. Notice how this compares to the expectation: it should agree to within numerical precision errors.

In [2]:
a=np.array([
    [4,-7,3],
    [1,3,-3],
    [3,-29,21]])
print(la.det(a),"This is 0 with the difference being cause by limited precision")


6.750155989720952e-14 This is 0 with the difference being cause by limited precision


Calculate and print the inverse of the matrix, $\mathsf{A}^{-1}$, using `scipy.linalg.inv`.  Store the inverse in a variable as we will use it below.

In [3]:
inva=la.inv(a)
print(inva)

[[ 5.68875742e+14 -1.42218936e+15 -2.84437871e+14]
 [ 7.11094678e+14 -1.77773670e+15 -3.55547339e+14]
 [ 9.00719925e+14 -2.25179981e+15 -4.50359963e+14]]


Based on the result from the previous part comment on why using $\mathsf{A}^{-1}$ in calculations will not produce numerically accurate results.

For a singular matrix, where the determinate is zero, the inverse of that matrix is undefined

Verify that the inverse calculated above is not really the inverse.  Do this by calculating both $\mathsf{A} \mathsf{A}^{-1}$ and $\mathsf{A}^{-1} \mathsf{A}$.  Compare these to the identity matrix, $\mathsf{I}$.  (*Note:* You can construct the identity matrix using `np.eye` or `np.identity` if you are so inclined.)

In [4]:
print(f"aa^-1: {a@inva}, \na^-1a: {inva@a}, \nI: {np.eye(3)}" )

aa^-1: [[ 2.    0.    0.25]
 [ 0.    0.   -0.25]
 [ 4.    0.    2.  ]], 
a^-1a: [[ 1.125 -1.     0.   ]
 [-0.25   2.    -2.   ]
 [ 0.25  -2.     2.   ]], 
I: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Briefly decribe the results from the previous part and what they mean.

Neither operation gave back the identity matrix, this means A is not invertible

Now that we have seen some of the problems with this matrix, let us look at its eigenvalues and eigenvectors.  We will discover that we *can reliably calculate* the eigenvalues and eigenvectors of a singular matrix.

Calculate and print the eigenvalues and the eigenvectors of the matrix $\mathsf{A}$.  In what follows I will call the matrix of eigenvectors $\mathsf{B}$. (*Note:* For comparison purposes the determinant of a matrix is the product of the eigenvalues of the matrix, that is 
$$\det(\mathsf{A}) = \prod_{j=1}^{N} \lambda_j$$
where the $\lambda_j$ are the eigenvalues of the $N\times N$ matrix $\mathsf{A}$.  Mathematically this means for a singular matrix that at least one of the eigenvalues must be zero.)

In [5]:
(lam, B)= la.eig(a)
print("Eigenvalues:", lam)
print("Matrix of eigenvectors:\n", B)
la.inv(B)@a@B

Eigenvalues: [6.66133815e-16+0.j 2.77502784e+00+0.j 2.52249722e+01+0.j]
Matrix of eigenvectors:
 [[ 0.44413993  0.89948293  0.17876267]
 [ 0.55517491  0.29539723 -0.12371167]
 [ 0.70322155  0.3219797   0.97608367]]


array([[ 1.31491833e-16, -7.04461456e-16,  3.74848479e-15],
       [ 6.66133815e-16,  2.77502784e+00,  1.55431223e-15],
       [-8.88178420e-15, -7.99360578e-15,  2.52249722e+01]])

The matrix of eigenvectors should diagonalize the matrix $\mathsf{A}$, in other words, $\mathsf{B}^{-1} \mathsf{A} \mathsf{B}$ should equal a diagonal matrix with the eigenvalues along the diagonal.  Verify this is true (to within machine precision error).  (*Note:* To construct a diagonal matrix from a vector you may use `np.diag`.)

In [6]:
assert np.allclose(la.inv(B)@a@B,np.diag(lam))


## Time Independent Schrödinger Equation

In the lab this week we will solve the time independent Schrödinger equation in one dimension for two different potentials.

Mathematically, much of quantum mechanics reduces to solving eigenvalue problems.  We have probably encountered the time independent Schrödinger equation in modern physics.  In its tersest form it may be written as $\hat H \psi(x)=E\psi(x)$.  Here $\hat H$ is the Hamiltonian operator which we will have more to say about, $E$ is the energy of the system, and $\psi(x)$ is the wave function which encodes the information about the state of the system.  The Hamiltonian is determined from the physics, so, given a Hamiltonian, our goal is to solve for the energy and the wave function.

When we first study quantum mechanics we typically write the Hamiltonian as a differential operator.  For a single particle in one dimension the nonrelativistic Hamiltonian may be written as
$$\hat H = -\frac{\hbar^2}{2m} \frac{\mathrm{d}^2}{\mathrm{d}x^2}+V(x).$$
Here $\hbar=h/2\pi$ is the reduced Planck constant and $V(x)$ is the potential energy (though we will just refer to it as a potential).  It is convenient to work in units of $\hbar$ and $m$ meaning that they get absorbed into the energies.  In this case the Schrödinger equation becomes
$$ -\frac12 \frac{\mathrm{d}^2 \psi(x)}{\mathrm{d}x^2}+V(x)\psi(x)=E\psi(x).$$

In a quantum mechanics course we solve this equation for various choices of the potential energy.  To solve it numerically we can proceed in a number of ways.  One is to use a differential equation integrator, such as `scipy.integrate.solve_ivp`, to do the work for us.  (Since this is such an important equation, specialized integrators have been developed which would be even better to use.)  **An alternative approach** is to instead turn it into a matrix equation and solve it as an eigenvalue problem.

As we very briefly mentioned when discussing numerical derivatives, we can discretize the second derivative using
$$ \frac{\mathrm{d}^2 \psi(x)}{\mathrm{d}x^2} = \frac{\psi(x-h) - 2 \psi(x) + \psi(x+h)}{h^2}.$$
Note that here $h$ is the step size and **has nothing to do with Planck’s constant**.  Despite the fact that this algorithm is only accurate to order $h^2$ it will be sufficient for our purposes.  With this we can rewrite the Schrödinger equation in matrix form

$$ \mathsf{H}\vec\psi = E\vec\psi, $$

where $\mathsf{H}$ is now **a matrix representing our Hamiltonian** and $\vec\psi$ is now **a vector representing our wave function**.

### Understanding the Linear System

We need to understand the linear system we just wrote down. The approach we will use and the structure we will find is very similar to that from PreLab 08.  To this end it is best to again grab some paper in order to work out some of the details.  We are again solving a one dimensional system in a region of length $L$ along the $x$-axis.  To do so, we discretize the wave function by evaluating it at $N+1$ points. Further, we will take these points to be equally spaced with step size $h$ along the $x$-axis so that $\psi_j \equiv \psi(x_j)$ with $x_j = x_0 + j h$.  With this and the discretized form of the second derivative we can rewrite $\hat H \psi(x)$ in the discrete (matrix) form $\mathsf{H}\vec \psi$.  Use this to find the form of $\mathsf{H}$.  You should find it is a symmetric tridiagonal matrix.

As in PreLab 08 it is best to begin with a small system.  Again let $N=4$ (so that we have 5 points).  Write out the 5 equations, one for each of the components $\psi_j$, and use this to construct the matrix $\mathsf{H}$.  Fill in the components of this matrix below.  For the boundary conditions let $\psi_{-1} = \psi_{N+1} = 0$.  Notationally it is also convenient to write $V_j\equiv V(x_j)$.

For your convenience, here is a template you can use to write the matrix.  Cut and paste this into the solution cell, then replace the dots with the appropriate expressions.

$$ \mathsf{H} = \begin{pmatrix}
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . 
\end{pmatrix} $$


$$ \mathsf{H} = \begin{pmatrix}
V(x_0)+1/h^2 & -\frac{1}{2h^2} & 0 & 0 & 0 \\
-\frac{1}{2h^2} & V(x_1)+1/h^2 & -\frac{1}{2h^2} & 0 & 0 \\
0 & -\frac{1}{2h^2} & V(x_2)+1/h^2 & -\frac{1}{2h^2} & 0 \\
0 & 0 & -\frac{1}{2h^2} & V(x_3)+1/h^2 & -\frac{1}{2h^2} \\
0 & 0 & 0 & -\frac{1}{2h^2} & V(x_4)+1/h^2
\end{pmatrix} $$

### Eigenvalue Problem

Now that we know $\mathsf{H}$ we can solve the eigenvalue problem $\mathsf{H}\vec\psi = E\vec\psi$ for $E$ and $\psi$.  Since $\mathsf{H}$ is a tridiagonal matrix it is again best to use a specialized function for this case.  Thus instead of using `scipy.linalg.eig` or `scipy.linalg.eigh` we will use `scipy.linalg.eig_banded`.  Look up its documentation.  You should see it is similar to what we encountered in the previous Lab except that it **requires a symmetric banded matrix**.  Fortunately that is exactly what we have.  In fact, this function is more similar to `scipy.linalg.solveh_banded`. 

It is easiest to use the function in its default configuration.  This means it will return both the eigenvalues and eigenvectors for $\mathsf{H}$ given the **upper** part of the matrix.  Once again study the documentation and pay particular attention to the *upper form* in the example.  There they show an example of a $6\times6$ matrix with 5 "diagonals" (2 upper and 2 lower bands along with the main diagonal.)

Provide code for constructing the matrix $\mathsf{H}$ in banded form as used by `scipy.linalg.eig_banded`.  Your code should work for arbitrary $N$, length $L$, and potential that will be input as a function and will be called as $V(x)$.

Write your function for constructing the Hamiltonian matrix below.

(*Note:* There are some elements of the matrix that are not used by the algorithm.  It is best to fill these in anyway.  In fact, it is easier to construct the matrix by not treating these elements as special.  Since there values do not matter, just fill them in along with the rest of the band in which they fall.)

In [7]:
def Hamiltonian (N, L, V):
    """ Returns ab, a banded matrix formatted to be used with sovle_banded, for H given size N, a function V and length L"""
    h=L/N
    # Construct the banded matrix
    ab = np.zeros((2,N+1))
    # Lower and upper bands are the same
    ab[0] = -1/(2*h**2)
    # Main diagonal
    ab[1] = V(np.arange(0,N))+1/h**2
    return ab


As a simple test case, if the potential is a constant, for example, $V(x)=1$, and we choose $L=1$ and $N=4$, then the matrix is simple to calculate. Remember that simple tests are a good start, but there are many types of errors it will not catch. 

In [8]:
assert(np.allclose(Hamiltonian(4, 1, lambda x: 1), 
                   np.array([[-8., -8., -8., -8., -8.],
                             [17., 17., 17., 17., 17.]])))

### Normalization

Unfortunately we are not quite done.  By convention, when we solve an eigenvalue problem the eigenvectors are returned as unit vectors.  This means that when we solve the Schrödinger equation as an eigenvalue problem the eigenvectors will satisfy $|\vec\psi|^2 = 1$.  However, in quantum mechanics to allow for a probabilistic interpretation of the wave function we instead normalize using the integral condition
$$ \int |\psi(x)|^2 \mathrm{d}x = 1.  $$
(Here the integral is over the entire region where $\psi(x)\ne 0$.)  **These are not the same conditions.**  This means we need to renormalize the eigenvector to be consistent with the requirement from quantum mechanics.

Given an eigenvector, $\vec\psi$, how must we modify it to be consistent with the normalization condition from quantum mechanics?  To determine this it is simplest to discretize the integral condition from quantum mechanics.  Here it is sufficient to use the Riemann sum form of the integral from calculus, with equally spaced points, and see what rescaling is required in order that the integral condition is satisfied.  In other word, solving the eigenvalue problem gives us a vector, let us call it $\vec\psi^{(\mathrm{ev})}$, that satisfies $|\vec\psi^{(\mathrm{ev})}|^2=1$.  The state relevant for quantum mechanics is proportional to this, $\vec\psi = \alpha \vec\psi^{(\mathrm{ev})}$.  Our objective is to find $\alpha$. 

appling the riemann sum:
$$  \sum_{i=0}^4 h |\psi(x_i)|^2=1 $$
$$   h |\psi(x_0)|^2 +  h |\psi(x_1)|^2  +  h |\psi(x_2)|^2 +  h |\psi(x_3)|^2 +  h |\psi(x_4)|^2=1 $$
$$  |\psi(x_0)|^2 + |\psi(x_1)|^2  +  |\psi(x_2)|^2 +  |\psi(x_3)|^2 +  |\psi(x_4)|^2=\frac{1}{h} $$
$$  \alpha^2 \vec{\psi}^{ev}^2 + \alpha^2 \vec{\psi}^{ev}^2  +  \alpha^2 \vec{\psi}^{ev}^2  + \alpha^2 \vec{\psi}^{ev}^2  +  \alpha^2 \vec{\psi}^{ev}^2 =\frac{1}{h} $$
$$  \alpha^2 (\vec{\psi}^{ev}^2 + \vec{\psi}^{ev}^2  +   \vec{\psi}^{ev}^2  +  \vec{\psi}^{ev}^2  +  \vec{\psi}^{ev}^2) =\frac{1}{h} $$
$$  \alpha^2 (1) =\frac{1}{h} $$
So:
$$ \alpha = \frac{1}{\sqrt{h}} $$

## Turning in the PreLab

All prelabs will be handled as was done for PreLab01.  See that file for details.  It will be assumed from now on that you have read and understood the procedure and what it means when you submit a prelab.